In [25]:
import numpy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical

# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
import numpy as np

In [ ]:
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    f = open(gloveFile,'r', encoding='utf8')
    model = {}
    for line in tqdm_notebook(f):
        splitLine = line.split(' ')
        word = splitLine[0]
        embedding = np.asarray(splitLine[1:], dtype='float32')
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model

# glove_model=loadGloveModel("D:/MyCodes/EMBEDDINGS/glove.840B.300d.txt")



In [ ]:
# import pickle
# with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","wb") as fh:
#     pickle.dump(glove_model,fh)
    

In [ ]:
import pickle
with open("D:/MyCodes/EMBEDDINGS/glove_840B_300d.pickle","rb") as fh:
    embed_dict=pickle.load(fh)


In [5]:
from tqdm import tqdm_notebook
from tqdm import tqdm

In [6]:
# Create embedding matrix

vocab=list(embed_dict.keys())




In [7]:
vocab.insert(0,"__unk__")


In [8]:
rev_vocab=dict((v,k) for k,v in enumerate(vocab))

In [37]:
with open("D:/SEM 7/Core Project/Dataset/Datasets/Classification_task/classification_clusters.pickle","rb") as fh:
    total_data=pickle.load(fh)

In [38]:
def create_labels(data):
    
    
    targets=[]
    
    for ques in tqdm_notebook(data):
        if ques[2].lower() not in targets:
            targets.append(ques[2].lower())
            
    return targets
    
    

In [39]:
uniq_targets=create_labels(total_data)
rev_targets=dict((v,k) for k,v in enumerate(uniq_targets))

HBox(children=(IntProgress(value=0, max=443757), HTML(value='')))

In [40]:

def encode_data(dictionary1,dictionary2,data):
    
    if dictionary1 ==None or dictionary2==None:
        print("Both dictionaries are not passed")
        return 
    
    encoded_totl_data=[]
    encoded_questions=[]
    encoded_targets=[]
    
    for ques in tqdm_notebook(data):
        
        if "?" in ques[0]:
            ques[0]=ques[0].replace("?"," ?")

        
        encoded_ques=[dictionary1.get(wrd.lower(),0) for wrd in (ques[0].split(" "))]  # Tokenizing and encoding
        encoded_targ=[dictionary2[ques[2].lower()]] # Encoding targets
        
        encoded_totl_data.append([encoded_ques,encoded_targ])
        encoded_questions.append(encoded_ques)
        encoded_targets.append(encoded_targ)
        
        
    return encoded_totl_data,encoded_questions,encoded_targets
    
    

In [41]:
encoded_total_data,encoded_questions,encoded_targets= encode_data(rev_vocab,rev_targets,total_data)


HBox(children=(IntProgress(value=0, max=443757), HTML(value='')))

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(encoded_questions,encoded_targets, test_size=0.2)
print( len(X_train), len(y_train))
print (len(X_test), len(y_test))


355005 355005
88752 88752


In [43]:
del encoded_total_data,encoded_questions,encoded_targets
del total_data

In [31]:
import numpy as np

# create a weight matrix for words in training docs

vocab_size=len(vocab)

embedding_matrix = np.zeros((vocab_size, 300))
for i,word in enumerate(vocab):
    embedding_vector = embed_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [44]:
X_train=np.array(X_train[:1])
X_test=np.array(X_test[:1])
y_train=np.array(y_train[:1])
y_test=np.array(y_test[:1])

In [49]:
y_train=to_categorical(y_train[:1],len(uniq_targets))
y_test=to_categorical(y_test[:1],len(uniq_targets))

In [50]:


embedding_vecor_length=300
# truncate and pad input sequences
max_review_length = 40
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length))
# model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_review_length, trainable=False))

model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 40, 32)            70272544  
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 70,325,845
Trainable params: 70,325,845
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

ValueError: Error when checking target: expected dense_6 to have shape (None, 1) but got array with shape (1, 4)

array([[0., 1., 0., 0.]])